# Best Engineering Practices

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/inefficient-networks/blob/master/docs/notebooks/mlops/04-deployment)
[![Stars](https://img.shields.io/github/stars/particle1331/inefficient-networks?style=social)](https://github.com/particle1331/inefficient-networks)

```text
𝗔𝘁𝘁𝗿𝗶𝗯𝘂𝘁𝗶𝗼𝗻: Notes for Module 6 of the MLOps Zoomcamp (2022) by DataTalks.Club.
```

---

## Introduction

In this module, we will cover best practices for developing and deploying our code. We will take our example [streaming code](https://particle1331.github.io/inefficient-networks/notebooks/mlops/04-deployment/notes.html#streaming-deploying-models-with-kinesis-and-lambda) from a previous module, break it down into testable units, and improve it with software engineering best practices. More precisely, we create and automate unit and integration testing, code quality checks, and add pre-commit hooks for these. We will also look at how to use `make` which is a nice tools for abstracting and automating repetitive but involved tasks. 

## Testing Python code with pytest

Let us look at [the script](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/04-deployment/streaming/lambda_function.py) that we will work on:

```python
import os
import json
import boto3
import base64

import mlflow


# Load environmental variables
PREDICTIONS_STREAM_NAME = os.getenv('PREDICTIONS_STREAM_NAME', 'ride_predictions')
RUN_ID = os.getenv('RUN_ID')
TEST_RUN = os.getenv('TEST_RUN', 'False') == 'True'

# Load model from S3
logged_model = f's3://mlflow-models-ron/1/{RUN_ID}/artifacts/model'
model = mlflow.pyfunc.load_model(logged_model)


def prepare_features(ride):
    features = {}
    features['PU_DO'] = '%s_%s' % (ride['PULocationID'], ride['DOLocationID'])
    features['trip_distance'] = ride['trip_distance']
    return features


def predict(features):
    pred = model.predict(features)
    return float(pred[0])


def lambda_handler(event, context):
    
    predictions_events = []
    
    for record in event['Records']:
        encoded_data = record['kinesis']['data']
        decoded_data = base64.b64decode(encoded_data).decode('utf-8')
        ride_event = json.loads(decoded_data)

        ride = ride_event['ride']
        ride_id = ride_event['ride_id']
    
        features = prepare_features(ride)
        prediction = predict(features)
    
        prediction_event = {
            'model': 'ride_duration_prediction_model',
            'version': '123',
            'prediction': {
                'ride_duration': prediction,
                'ride_id': ride_id
            }
        }

        if not TEST_RUN:
            kinesis_client = boto3.client('kinesis')
            kinesis_client.put_record(
                StreamName=PREDICTIONS_STREAM_NAME,
                Data=json.dumps(prediction_event),
                PartitionKey=str(ride_id)
            )
        
        predictions_events.append(prediction_event)


    return {
        'predictions': predictions_events
    }
```

To review, first this script loads the environmental variables and the model from S3. Then, it defines two helper functions for preprocessing and making prediction with the model. The most important function in this script is `lambda_handler` which takes in an `event` which contains a batch of events from the input stream. This explains the outer loop over `event['Records']`. 

Inside this block, the data is decoded and a prediction is made which is packaged as an event for the output stream. If this function is in production, i.e. outside of a test run, then the prediction event is written on the output stream. In this case, a Kinesis client is instantiated, and an output event is written to the specific predictions stream.

### Addding unit tests

First, we will create a `tests/` folder where we will put all our tests. We will be using `pipenv` to manage our environment. See the [previous module](https://particle1331.github.io/inefficient-networks/notebooks/mlops/04-deployment/notes.html#setting-up-the-environment-with-pipenv) for details. We will start with the following Pipfile:

```ini
[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
boto3 = "*"
mlflow = "*"
scikit-learn = "==1.0.2"

[dev-packages]
pytest = "*"

[requires]
python_version = "3.9"
```

Notice that this installs [pytest](https://docs.pytest.org/en/7.1.x/) as a dev dependency. Let us create one test:

```python
# tests/model_test.py
import lambda_function


def test_prepare_features():
    """Test preprocessing."""

    ride = {
        "PULocationID": 140,
        "DOLocationID": 205,
        "trip_distance": 2.05
    }

    actual_features = lambda_function.prepare_features(ride)
    
    expected_features = {
        'PU_DO': '140_205',
        'trip_distance': 2.05,
    }

    assert actual_features == expected_features
```

Before running this, since this test is not related to the model, we can comment out the block that loads the model from S3 to make this test run faster. Tests can be run either by doing `$ pytest` on the terminal:

```bash
$ pytest
======================== test session starts ========================
platform darwin -- Python 3.9.12, pytest-7.1.2, pluggy-1.0.0
rootdir: /Users/particle1331/code/inefficient-networks/docs/notebooks/mlops/06-best-practices
plugins: anyio-3.6.1
collected 1 item

tests/model_test.py .                                         [100%]

========================= 1 passed in 1.03s =========================
```

Or using the UI in VS Code after selecting pytest in the configuration:

```{figure} ../../../img/vscode-testing.png
---
width: 40em
---
Testing interface in VS Code. Really convenient for running and visualizing tests.
```

Note that we had to manually comment out things in our tests. This is not really great. Also, it would fail if our dev environment cannot connect to S3. 

Initial version. Focus on prediction. No writing on stream.

```python
import os
import model

RUN_ID = os.getenv('RUN_ID')

model_service = model.init(run_id=RUN_ID)


def lambda_handler(event, context):
    # pylint: disable=unused-argument
    return model_service.lambda_handler(event)
```

```python
import json
import mlflow
import base64


def load_model(run_id):
    model_path = f's3://mlflow-models-ron/1/{run_id}/artifacts/model'
    model = mlflow.pyfunc.load_model(model_path)
    return model


def base64_decode(encoded_data):
    decoded_data = base64.b64decode(encoded_data).decode('utf-8')
    ride_event = json.loads(decoded_data)
    return ride_event


class ModelService:

    def __init__(self, model, model_version=None):
        self.model = model
        self.model_version = model_version

    def prepare_features(self, ride):
        features = {}
        features['PU_DO'] = f"{ride['PULocationID']}_{ride['DOLocationID']}"
        features['trip_distance'] = ride['trip_distance']
        return features

    def predict(self, features):
        pred = self.model.predict(features)
        return float(pred[0])

    def lambda_handler(self, event):
        """Predict on batch of input events."""

        predictions_events = []

        for record in event['Records']:

            # Decode data from input kinesis stream
            encoded_data = record['kinesis']['data']
            ride_event = base64_decode(encoded_data)

            # Pickout id to match input to output
            ride = ride_event['ride']
            ride_id = ride_event['ride_id']

            # Make predictions using model
            features = self.prepare_features(ride)
            prediction = self.predict(features)

            # Package prediction event for output stream
            prediction_event = {
                'model': 'ride_duration_prediction_model',
                'version': self.model_version,
                'prediction': {'ride_duration': prediction, 'ride_id': ride_id},
            }

            predictions_events.append(prediction_event)

        return {'predictions': predictions_events}


def init(run_id: str):
    """Initialize model service."""

    model = load_model(run_id)
    model_service = ModelService(model=model, model_version=run_id)

    return model_service
```

```
docker build -t stream-model-duration:v2 .
```

```
docker run -it --rm -p 8080:8080 --env-file .env stream-model-duration:v2
```

python test_docker.py to check if still working.

test base64 decode
test prepare_features

```python
from model import ModelService
from model import base64_decode


def test_base64_decode():
    base64_input = "eyAgICAgICAgICAicmlkZSI6IHsgICAgICAgICAgICAgICJQVUxvY2F0aW9uSUQiOiAxMzAsICAgICAgICAgICAgICAiRE9Mb2NhdGlvbklEIjogMjA1LCAgICAgICAgICAgICAgInRyaXBfZGlzdGFuY2UiOiAzLjY2ICAgICAgICAgIH0sICAgICAgICAgICJyaWRlX2lkIjogMTIzICAgICAgfQ=="

    actual_result = base64_decode(base64_input)
    expected_result = {
        "ride": {
            "PULocationID": 130,
            "DOLocationID": 205,
            "trip_distance": 3.66,
        },
        "ride_id": 123,
    }

    assert actual_result == expected_result


def test_prepare_features():
    """Test preprocessing."""

    ride = {
        "PULocationID": 140,
        "DOLocationID": 205,
        "trip_distance": 2.05
    }

    model_service = ModelService(model=None)

    actual_features = model_service.prepare_features(ride)
    
    expected_features = {
        'PU_DO': '140_205',
        'trip_distance': 2.05,
    }

    assert actual_features == expected_features

```

refactor to make easier to testm

model mock = test model same methods and attributes / stupid model

adding callbacks

tdd, every change -> code still works

these are unit tests -> test individual functionalities

integration test -> make sure runs correctly as a whole


unit tests quite limited

not really care about accuracy of model, just want to make sure flow of prediction works. so we want to remove dependency on s3. remove all hard coding

```bash
aws s3 ls
```